In [ ]:
import matplotlib
import matplotlib.pyplot as plt 
import numpy as np
import pandas as pd
import os
import umap
import datashader as ds
import colorcet as cc
import igraph
import tqdm
from scipy import sparse
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.cluster import KMeans

from matplotlib.collections import PolyCollection
from matplotlib.colors import ListedColormap

from dredFISH.Analysis import TissueGraph
from dredFISH.Visualization import Viz
from dredFISH.Utils.__init__plots import * 
from dredFISH.Utils import powerplots
from dredFISH.Utils import miscu
from dredFISH.Utils import tmgu

import importlib
importlib.reload(Viz)
importlib.reload(TissueGraph)

In [ ]:
# %%time
# # slow for 24 bits; fast for 2D
# k = 30 
# NN = NearestNeighbors(n_neighbors=k)
# NN.fit(XY)
# knn = NN.kneighbors(XY, return_distance=False)

# knn
# # use pynndescent


In [ ]:
# %%time
# n_topics_list = [2,5,10]
# n_procs = 3 

# topic_cls = Classification.TopicClassifier(TMG.Layers[0])
# topic_cls.train(n_topics_list=n_topics_list, n_procs=n_procs)
# topics = topic_cls.classify(topic_cls.Env)

# Key lines
# n_topics = 10
# lda = LatentDirichletAllocation(n_components=n_topics)
# B = lda.fit(env)
# T = lda.transform(Env)

#### Load data

In [ ]:
respath = '/bigstore/GeneralStorage/fangming/projects/dredfish/figures/'

In [ ]:
basepth = '/bigstore/GeneralStorage/Data/dredFISH/Dataset1'
!ls -alhtr $basepth
!head $basepth"/TMG.json"

In [ ]:
df = pd.read_csv(
    os.path.join(basepth, "analysis_dev.csv"))
df

In [ ]:
TMG = TissueGraph.TissueMultiGraph(basepath=basepth, 
                                   redo=False, # load existing 
                                  )
TMG

In [ ]:
# spatial coordinates
layer = TMG.Layers[0]
XY = layer.XY
x, y = XY[:,0], XY[:,1]
###
x, y = y, x # a temporary hack
###

cells = layer.adata.obs.index.values

N = layer.N
print(N)
# measured basis
ftrs_mat = layer.feature_mat

# umap_mat = umap.UMAP(n_neighbors=30, min_dist=0.1).fit_transform(ftrs_mat)

# types

# regions

In [ ]:
labels = df['type_r0.1'].values
edges = np.asarray(layer.SG.get_edgelist()) 
ctg, ctg_idx = np.unique(labels, return_inverse=True) 
print(ctg)

i = edges[:,0] # cells
j = ctg_idx[edges[:,1]] # types it connects
dat = np.repeat(1, len(i))

env_mat = sparse.coo_matrix((dat, (i,j)), shape=(N, len(ctg))).toarray() # dense
env_mat = env_mat/env_mat.sum(axis=1).reshape(-1,1)
env_mat = np.nan_to_num(env_mat, 0)
env_mat

In [ ]:
%%time
k_kms = [2,5,10,20,50,100]
for k_km in tqdm.tqdm(k_kms):
    kmeans = KMeans(n_clusters=k_km)
    reg_clsts = kmeans.fit_predict(env_mat)
    df[f'type_reg_k{k_km}'] = np.char.add('t', np.array(reg_clsts).astype(str))

In [ ]:
for k_km in k_kms:
    hue = f'type_reg_k{k_km}'
    hue_order = np.sort(np.unique(df[hue]))
    ntypes = len(hue_order)

    fig, axs = plt.subplots(1, 2, figsize=(8*2,6))
    fig.suptitle(f"{hue}; n={ntypes}")
    ax = axs[0]
    sns.scatterplot(data=df, x='x', y='y', 
                    hue=hue, hue_order=hue_order, 
                    s=0.5, edgecolor=None, 
                    legend=False,
                    ax=ax)
    # ax.legend(loc='upper left', bbox_to_anchor=(0, -0.1), ncol=5)
    ax.set_aspect('equal')
    ax.axis('off')

    ax = axs[1]
    sns.scatterplot(data=df, x='umap_x', y='umap_y', 
                    hue=hue, hue_order=hue_order, 
                    s=0.5, edgecolor=None, 
                    legend=False,
                    ax=ax)
    # ax.legend(loc='upper left', bbox_to_anchor=(0, -0.1), ncol=5)
    ax.set_aspect('equal')
    ax.axis('off')
    fig.subplots_adjust(wspace=0)
    plt.show()

# Delaunay k neighbor analysis

In [ ]:
dgr = layer.SG.indegree() # same as out

In [ ]:
sns.histplot(dgr)
plt.xlabel('Number of Delaunay neighbors')
plt.ylabel('PDF')

In [ ]:
plt.plot(np.sort(dgr), np.linspace(0,1,len(dgr)))
plt.xlabel('Number of Delaunay neighbors')
plt.ylabel('CDF')

# Get from region types to regions; and visualize region boundaries 

In [ ]:
k_km = 10
region_types = df[f'type_reg_k{k_km}'].values
region_types

In [ ]:
polys = TMG.Geoms[0]['poly'] #) #) #[0]
bdbox = np.array(TMG.Geoms[0]['BoundingBox'].exterior.xy).T

# a hack
polys = [np.vstack([poly[:,1], poly[:,0]]).T for poly in polys]
bdbox = np.array(np.vstack([bdbox[:,1], bdbox[:,0]])).T
# end of the hack

mx = np.max(bdbox,axis=0)
mn = np.min(bdbox,axis=0)

ctg, idx = np.unique(region_types, return_inverse=True)
colors = sns.color_palette("tab10", len(ctg))
colors

In [ ]:
fig, ax = plt.subplots(figsize=(12,10))
p = PolyCollection(polys, edgecolors=('none'), cmap=ListedColormap(colors)) # cmap=self.clrmp)
p.set_array(idx)

ax.add_collection(p)
ax.set_aspect('equal') #, 'box')
ax.set_xlim([mn[0],mx[0]])
ax.set_ylim([mn[1],mx[1]])
ax.axis('off')

plt.show()

# get regions (region type zones) 

In [ ]:
# # get SG
# SG = layer.SG

# # trim edges -- remove connects from diffent types
# edges = np.asarray(SG.get_edgelist())
# edges_bytype = region_types[edges]
# edges_sametype = edges[edges_bytype[:,0]==edges_bytype[:,1]]
    
# # get components (same type and spatially connected); each component is assigned an index
# zones = igraph.Graph(n=N, edges=edges_sametype, directed = False)
# zone_ids = np.asarray(zones.components().membership)
# zone_ids

# # how do you visualize it?

# LDA

In [ ]:
%%time
lda = LatentDirichletAllocation(n_components=5)
y5 = lda.fit_transform(env_mat)
y5

In [ ]:
%%time
lda = LatentDirichletAllocation(n_components=10)
y10 = lda.fit_transform(env_mat)
y10

In [ ]:
y10d = np.argmax(y10, axis=1)
y5d = np.argmax(y5, axis=1)
np.unique(y10d)

In [ ]:
region_types = y5d
polys = TMG.Geoms[0]['poly'] #) #) #[0]
bdbox = np.array(TMG.Geoms[0]['BoundingBox'].exterior.xy).T

# a hack
polys = [np.vstack([poly[:,1], poly[:,0]]).T for poly in polys]
bdbox = np.array(np.vstack([bdbox[:,1], bdbox[:,0]])).T
# end of the hack

mx = np.max(bdbox,axis=0)
mn = np.min(bdbox,axis=0)

ctg, idx = np.unique(region_types, return_inverse=True)
colors = sns.color_palette("tab10", len(ctg))
colors

In [ ]:
fig, ax = plt.subplots(figsize=(12,10))
p = PolyCollection(polys, edgecolors=('none'), cmap=ListedColormap(colors)) # cmap=self.clrmp)
p.set_array(idx)

ax.add_collection(p)
ax.set_aspect('equal') #, 'box')
ax.set_xlim([mn[0],mx[0]])
ax.set_ylim([mn[1],mx[1]])
ax.axis('off')

plt.show()

# TMG related stuff (can't run; ref only)

In [ ]:
def contract_graph(self, TypeVec=None):
    """find zones/region, i.e. spatially continous areas in the graph the same (cell/microenvironment) type

    reduce graph size by merging spatial neighbors of same type. 
    Given a vector of types, will contract the graph to merge vertices that are both next to each other and of the same type. 

    Parameters
    ----------
    TypeVec : 1D numpy array with dtype int (default value is self.Type)
        a vector of Types for each node. If None, will use self.Type

    Note
    ----
    Default behavior is to assign the contracted TG the same taxonomy as the original graph. 

    Returns
    -------
    TissueGraph 
        A TG object after vertices merging. 
    """

    # Figure out which type to use
    if TypeVec is None: 
        TypeVec = self.Type

    # get edge list - work with names and not indexes in case things shift around (they shouldn't),     
    EL = np.asarray(self.SG.get_edgelist()).astype("int")
    nm = self.adata.obs["name"]
    EL[:,0] = np.take(nm,EL[:,0])
    EL[:,1] = np.take(nm,EL[:,1])

    # only keep edges where neighbors are of same types
    EL = EL[np.take(TypeVec,EL[:,0]) == np.take(TypeVec,EL[:,1]),:]

    # remake a graph with potentially many components
    IsoZonesGraph = igraph.Graph(n=self.N, edges=EL, directed = False)
    IsoZonesGraph = IsoZonesGraph.as_undirected().simplify()

    # because we used both type and proximity, the original graph (based only on proximity)
    # that was a single component graph will be broken down to multiple components 
    # finding clusters for each component. 
    cmp = IsoZonesGraph.components()

    IxMapping = np.asarray(cmp.membership)

    ZoneName, ZoneSingleIx = np.unique(IxMapping, return_index=True) # be cautious

    # zone size sums the current graph zone size per each aggregate (i.e. zone or microenv)
    df = pd.DataFrame(data = self.node_size)
    df['type'] = IxMapping
    ZoneSize = df.groupby(['type']).sum()
    ZoneSize = np.array(ZoneSize).flatten()

    # calculate zones feature_mat
    # if all values are Nan, just replace with tuple of the required size
    if np.all(np.isnan(self.feature_mat)): 
        zone_feat_mat = (len(ZoneSize),self.feature_mat.shape[1])
    else: 
        df = pd.DataFrame(data = self.feature_mat)
        df['type']=IxMapping
        zone_feat_mat = np.array(df.groupby(['type']).mean())

    # create new SG for zones 
    ZSG = self.SG.copy()

    comb = {"X" : "mean",
           "Y" : "mean",
           "Type" : "ignore",
           "name" : "ignore"}

    ZSG.contract_vertices(IxMapping,combine_attrs=comb)
    ZSG.simplify()

    # create a new Tissue graph by copying existing one, contracting, and updating XY
    ZoneGraph = TissueGraph(feature_mat=zone_feat_mat, 
                            basepath=self.basepath,
                            layer_type="isozone",
                            redo=True,
                            )

    ZoneGraph.SG = ZSG
    ZoneGraph.names = ZoneName
    ZoneGraph.node_size = ZoneSize
    ZoneGraph.Type = TypeVec[ZoneSingleIx]
    ZoneGraph.Upstream = IxMapping
    ZoneGraph.tax = self.tax

    return(ZoneGraph)

In [ ]:
# TMG.create_region_layer(topics, topic_cls.tax)
# # logging.info(f"TMG has {len(TMG.Layers)} Layers")

# create region layers through graph contraction
CG = self.Layers[cell_layer].contract_graph(topics) # contract the cell layer by topics

# contraction assumes that the feature_mat and taxonomy of the contracted layers are
# inherited from the layer used for contraction. This is not true for regions so we need to update these
# feature_mat is updated here and tax is updated by calling add_type_information
Env = self.Layers[cell_layer].extract_environments(typevec=CG.Upstream)
row_sums = Env.sum(axis=1)
row_sums = row_sums[:,None]
Env = Env/row_sums

# create the region layer merging information from contracted graph and environments
RegionLayer = TissueGraph(feature_mat=Env, basepath=self.basepath, layer_type="region", redo=True)
RegionLayer.SG = CG.SG.copy()
RegionLayer.node_size = CG.node_size.copy()
RegionLayer.Upstream = CG.Upstream.copy()

self.Layers.append(RegionLayer)
current_layer_id = len(self.Layers)-1
self.add_type_information(current_layer_id, CG.Type, region_tax)

# update the layers graph to show that regions are created from cells
self.layers_graph.append((cell_layer, current_layer_id))

In [ ]:

def plot_polys(self): 
    p = PolyCollection(self.V.TMG.Geoms[self.section]['poly'], cmap=self.clrmp)
    p.set_array(self.Styles['polygon']['scalar']) # a scalar -- color?
    self.ax.add_collection(p)